In [1]:
import pandas as pd, numpy as np

In [2]:
#Loading the datasets
grant = pd.read_csv('Grant.csv',header = None)
publication_1 = pd.read_csv('Publication_1.csv', header = None)
publication_3 = pd.read_csv('Publication_3.csv', header = None)

In [3]:
#Selecting relevant features
grant = grant.iloc[1:, :]
grant.columns = grant.iloc[0]
grant = grant.drop(grant.index[0])
publication_1 = publication_1.iloc[1:,:]
publication_1.columns = publication_1.iloc[0]
publication_1 = publication_1[['PMID','Title','Abstract','Publication Date (print)','Authors','Authors (Raw Affiliation)'
                               ,'Corresponding Authors','Research Organizations - standardized',
                               'Country of Research organization','Funder Group', 'Times cited', 'Altmetric',
                              'Sustainable Development Goals']]
publication_1 = publication_1.drop(publication_1.index[0])
publication_3 = publication_3.iloc[1:,:]
publication_3.columns = publication_3.iloc[0]
publication_3 = publication_3[['PMID','Title','Abstract','Publication Date (print)','Authors','Authors (Raw Affiliation)'
                               ,'Corresponding Authors','Research Organizations - standardized',
                               'Country of Research organization','Funder Group', 'Times cited', 'Altmetric',
                              'Sustainable Development Goals']]
publication_3 = publication_3.drop(publication_3.index[0])

In [5]:
#Merging the two publication dataset
df_diff = publication_1[~publication_1.PMID.isin(publication_3.PMID)]
publication_df = pd.concat([publication_3, df_diff], ignore_index= True)
publication_df.shape

(6920, 13)

In [6]:
#Loading the altmetric (larger) dataset
altmetric_df = pd.read_csv('Altmetric - Research Outputs - Texas A&M University - 2022-03-12.csv',encoding='cp1252')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#Selecting relevant features
altmetric_df = altmetric_df[['Altmetric Attention Score','Title', 'Authors at my Institution','Departments','Funder',
                             'Publication Date','PubMed ID','News mentions','Blog mentions','Policy mentions','Patent mentions',
                             'Twitter mentions','Peer review mentions','Weibo mentions',
                             'Facebook mentions','Wikipedia mentions','Google+ mentions','LinkedIn mentions',
                             'Reddit mentions','Pinterest mentions','F1000 mentions',
                            'Q&A mentions','Video mentions','Syllabi mentions','Number of Mendeley readers',
                             'Number of Dimensions citations']]
altmetric_df = altmetric_df.rename(columns = {'PubMed ID': "PMID"})

In [8]:
#Typecasting PMID to int while null values are marked 0
altmetric_df['PMID'] = altmetric_df['PMID'].fillna(0)
altmetric_df['PMID'] = altmetric_df['PMID'].astype(int)

In [9]:
#Typecasting PMID to int and null values are marked 1
publication_df['PMID'] = publication_df['PMID'].fillna(1).astype('int')


In [10]:
#Combining the dataframes, taking care of similar columns and null values by taking a union
combined_df = pd.merge(altmetric_df, publication_df, on = ['PMID'], how = 'outer').set_index('PMID')
combined_df['Publication Date (print)'].fillna(combined_df['Publication Date'],inplace = True)
combined_df['Publication Date'].fillna(combined_df['Publication Date (print)'], inplace = True)
combined_df['Title_y'].fillna(combined_df['Title_x'],inplace= True)
combined_df['Title_x'].fillna(combined_df['Title_y'],inplace= True)
combined_df['Altmetric Attention Score'].fillna(combined_df['Altmetric'],inplace = True)
combined_df['Altmetric'].fillna(combined_df['Altmetric Attention Score'],inplace = True)
combined_df = combined_df[combined_df['Publication Date'].notna()]
combined_df.pop('Title_y')
combined_df.pop('Altmetric')
combined_df.pop('Publication Date (print)')
combined_df['Title'] = combined_df['Title_x']
combined_df.pop('Title_x')
combined_df

,Altmetric Attention Score,Authors at my Institution,Departments,Funder,Publication Date,News mentions,Blog mentions,Policy mentions,Patent mentions,Twitter mentions,...,Abstract,Authors,Authors (Raw Affiliation),Corresponding Authors,Research Organizations - standardized,Country of Research organization,Funder Group,Times cited,Sustainable Development Goals,Title
PMID,,,,,,,,,,,,,,,,,,,,,
32527856,21128,"ZHANG, Renyi",Atmospheric Sciences; Texas A&M; College of Ge...,Welch Foundation,11/06/2020,719.0,47.0,3.0,1.0,37528.0,...,Various mitigation measures have been implemen...,"Zhang, Renyi; Li, Yixin; Zhang, Annie L; Wang,...","Zhang, Renyi (Department of Atmospheric Scienc...","Zhang, Renyi (Texas A&M University; Texas A&M ...",Texas A&M University; California Institute of ...,United States; United States; United States; U...,NaN,652,NaN,Identifying airborne transmission as the domin...
28864332,7685,"KUMAR, Rajesh",Texas A&M; Medical Physiology; College of Medi...,Canadian Institutes of Health Research; Colcie...,11/1/2017,315.0,35.0,3.0,0.0,12139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Associations of fats and carbohydrate intake w...
30518902,5670,"XU, Yangyang",Atmospheric Sciences; College of Geosciences; ...,NaN,12/5/2018,95.0,17.0,0.0,0.0,9548.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Global warming will happen faster than we think
31569235,4504,"STOVER, Patrick J; JOHNSTON, Bradley C",College of Agriculture and Life Sciences; Epid...,National Institute of Environmental Health Sci...,10/1/2019,510.0,47.0,0.0,0.0,3211.0,...,This article has been corrected. The original ...,"Johnston, Bradley C; Zeraatkar, Dena; Han, Mi ...","Johnston, Bradley C (Dalhousie University, Hal...",NaN,Hospital of the Holy Cross and Saint Paul; Net...,Spain; Netherlands; Canada; Canada; South Kore...,ICRP - International Cancer Research Partnersh...,126,NaN,Unprocessed Red Meat and Processed Meat Consum...
26315443,4156,"DONNELLAN, Michael B; THOMAS, Shawna L",Texas A&M; Computer Science and Engineering; I...,Laura and John Arnold Foundation; Medical Rese...,8/28/2015,364.0,238.0,6.0,0.0,2402.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Estimating the reproducibility of psychologica...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8447477,NaN,NaN,NaN,NaN,1993-02-01,NaN,NaN,NaN,NaN,NaN,...,The influence of flow on endothelial intracell...,"Falcone, J. C.; Kuo, L.; Meininger, G. A.","Falcone, J. C. (Department of Medical Physiolo...",NaN,Texas A&M University,United States,ICRP - International Cancer Research Partnersh...,124,NaN,Endothelial cell calcium increases during flow...
11287318,NaN,NaN,NaN,NaN,2001-05-01,NaN,NaN,NaN,NaN,NaN,...,Ras proteins are critical regulators of cell f...,"Collett, Esther D.; Davidson, Laurie A.; Fan, ...","Collett, Esther D. (Molecular and Cell Biology...",NaN,Texas A&M University,United States,ICRP - International Cancer Research Partnersh...,124,NaN,n-6 and n-3 polyunsaturated fatty acids differ...
18354641,3,NaN,NaN,NaN,2000-12-01,NaN,NaN,NaN,NaN,NaN,...,Mueller matrices provide a complete characteri...,"Jiao, S; Yao, G; Wang, L V","Jiao, S (Optical Imaging Laboratory, Biomedica...",NaN,Texas A&M University,United States,NaN,124,NaN,Depth-resolved two-dimensional stokes vectors ...


In [11]:
#Saving the file as csv
#combined_df.to_csv('combined_df.csv')

In [12]:
combined_df = pd.read_csv('combined_df.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3,4,25,26,27,28,29,30,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
test_list = combined_df['Departments'].str.split(';').reset_index(drop = True)
test_list

0         [Atmospheric Sciences,  Texas A&M,  College of...
1         [Texas A&M,  Medical Physiology,  College of M...
2         [Atmospheric Sciences,  College of Geosciences...
3         [College of Agriculture and Life Sciences,  Ep...
4         [Texas A&M,  Computer Science and Engineering,...
                                ...                        
182876                                                  NaN
182877                                                  NaN
182878                                                  NaN
182879                                                  NaN
182880                                                  NaN
Name: Departments, Length: 182881, dtype: object

In [14]:
stopwords = ['College', 'Texas','Center','School','Institute', 'Division']
for i in range(test_list.shape[0]):
    try:
        temp_lst = test_list[i].copy()
        for ele in temp_lst:
            for bad_word in stopwords:
                if bad_word in ele:
                    test_list[i].remove(ele)
    except:
        continue
test_list

0                                    [Atmospheric Sciences]
1                                     [ Medical Physiology]
2                                    [Atmospheric Sciences]
3                         [ Epidemiology and Biostatistics]
4         [ Computer Science and Engineering,  Psycholog...
                                ...                        
182876                                                  NaN
182877                                                  NaN
182878                                                  NaN
182879                                                  NaN
182880                                                  NaN
Name: Departments, Length: 182881, dtype: object

In [15]:
primary_df = combined_df.drop(['Authors', 'Authors (Raw Affiliation)',
       'Corresponding Authors','Number of Dimensions citations'],axis = 1).reset_index(drop = False)
primary_df['Departments'] = test_list
primary_df

,index,PMID,Altmetric Attention Score,Authors at my Institution,Departments,Funder,Publication Date,News mentions,Blog mentions,Policy mentions,...,Video mentions,Syllabi mentions,Number of Mendeley readers,Abstract,Research Organizations - standardized,Country of Research organization,Funder Group,Times cited,Sustainable Development Goals,Title
0,0,32527856,21128.0,"ZHANG, Renyi",[Atmospheric Sciences],Welch Foundation,11/06/2020,719.0,47.0,3.0,...,6.0,0.0,1245.0,Various mitigation measures have been implemen...,Texas A&M University; California Institute of ...,United States; United States; United States; U...,NaN,652.0,NaN,Identifying airborne transmission as the domin...
1,1,28864332,7685.0,"KUMAR, Rajesh",[ Medical Physiology],Canadian Institutes of Health Research; Colcie...,11/1/2017,315.0,35.0,3.0,...,30.0,0.0,1996.0,NaN,NaN,NaN,NaN,NaN,NaN,Associations of fats and carbohydrate intake w...
2,2,30518902,5670.0,"XU, Yangyang",[Atmospheric Sciences],NaN,12/5/2018,95.0,17.0,0.0,...,3.0,0.0,228.0,NaN,NaN,NaN,NaN,NaN,NaN,Global warming will happen faster than we think
3,3,31569235,4504.0,"STOVER, Patrick J; JOHNSTON, Bradley C",[ Epidemiology and Biostatistics],National Institute of Environmental Health Sci...,10/1/2019,510.0,47.0,0.0,...,14.0,0.0,415.0,This article has been corrected. The original ...,Hospital of the Holy Cross and Saint Paul; Net...,Spain; Netherlands; Canada; Canada; South Kore...,ICRP - International Cancer Research Partnersh...,126.0,NaN,Unprocessed Red Meat and Processed Meat Consum...
4,4,26315443,4156.0,"DONNELLAN, Michael B; THOMAS, Shawna L","[ Computer Science and Engineering, Psycholog...",Laura and John Arnold Foundation; Medical Rese...,8/28/2015,364.0,238.0,6.0,...,2.0,0.0,4592.0,NaN,NaN,NaN,NaN,NaN,NaN,Estimating the reproducibility of psychologica...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182876,182876,8447477,NaN,NaN,NaN,NaN,1993-02-01,NaN,NaN,NaN,...,NaN,NaN,NaN,The influence of flow on endothelial intracell...,Texas A&M University,United States,ICRP - International Cancer Research Partnersh...,124.0,NaN,Endothelial cell calcium increases during flow...
182877,182877,11287318,NaN,NaN,NaN,NaN,2001-05-01,NaN,NaN,NaN,...,NaN,NaN,NaN,Ras proteins are critical regulators of cell f...,Texas A&M University,United States,ICRP - International Cancer Research Partnersh...,124.0,NaN,n-6 and n-3 polyunsaturated fatty acids differ...
182878,182878,18354641,3.0,NaN,NaN,NaN,2000-12-01,NaN,NaN,NaN,...,NaN,NaN,NaN,Mueller matrices provide a complete characteri...,Texas A&M University,United States,NaN,124.0,NaN,Depth-resolved two-dimensional stokes vectors ...
182879,182879,17776922,NaN,NaN,NaN,NaN,1977-06-24,NaN,NaN,NaN,...,NaN,NaN,NaN,A 400-square-kilometer depression in the conti...,Texas A&M University,United States,NaN,124.0,NaN,"Anoxic, Hypersaline Basin in the Northern Gulf..."


In [51]:
#primary_df.to_csv('primary_df.csv')

In [16]:
#primary_df = pd.read_csv('primary_df.csv').iloc[:,2:]
#primary_df

In [17]:
"""dept_df = primary_df.dropna(subset=['Departments'])[['Authors at my Institution','Departments','Publication Date',
                                                    'Abstract','Research Organizations - standardized',
                                                    'Country of Research organization','Times cited',
                                                    'Title']]
dept_df = dept_df[dept_df['Departments']!='[]'].reset_index().iloc[:,1:]
dept_df"""

"dept_df = primary_df.dropna(subset=['Departments'])[['Authors at my Institution','Departments','Publication Date',\n                                                    'Abstract','Research Organizations - standardized',\n                                                    'Country of Research organization','Times cited',\n                                                    'Title']]\ndept_df = dept_df[dept_df['Departments']!='[]'].reset_index().iloc[:,1:]\ndept_df"

In [64]:
#dept_df.to_csv('primary_df.csv')
dept_df = pd.read_csv('primary_df_2.csv').iloc[:,1:]

In [65]:
dept_df

,Authors at my Institution,Departments,Publication Date,Research Organizations - standardized,Country of Research organization,Times cited,Title,Title_abs
0,"ZHANG, Renyi",['Atmospheric Sciences'],2020,Texas A&M University; California Institute of ...,United States; United States; United States; U...,652.0,Identifying airborne transmission as the domin...,Identifying airborne transmission as the domin...
1,"KUMAR, Rajesh",[' Medical Physiology'],2017,NaN,NaN,NaN,Associations of fats and carbohydrate intake w...,Associations of fats and carbohydrate intake w...
2,"XU, Yangyang",['Atmospheric Sciences'],2018,NaN,NaN,NaN,Global warming will happen faster than we think,Global warming will happen faster than we think
3,"STOVER, Patrick J; JOHNSTON, Bradley C",[' Epidemiology and Biostatistics'],2019,Hospital of the Holy Cross and Saint Paul; Net...,Spain; Netherlands; Canada; Canada; South Kore...,126.0,Unprocessed Red Meat and Processed Meat Consum...,Unprocessed Red Meat and Processed Meat Consum...
4,"DONNELLAN, Michael B; THOMAS, Shawna L","[' Computer Science and Engineering', ' Psycho...",2015,NaN,NaN,NaN,Estimating the reproducibility of psychologica...,Estimating the reproducibility of psychologica...
...,...,...,...,...,...,...,...,...
160358,"BEHMER, Spencer T",[' Entomology'],2012,NaN,NaN,NaN,Plant sterols and host plant suitability for g...,Plant sterols and host plant suitability for g...
160359,"KHASAWNEH, Fadi",[' Pharmacy Practice'],2011,NaN,NaN,NaN,Glutathione and Adaptive Immune Responses agai...,Glutathione and Adaptive Immune Responses agai...
160360,"KAISER, Karl",[' Marine Sciences'],2011,NaN,NaN,NaN,Characterization of Lignin by Gas Chromatograp...,Characterization of Lignin by Gas Chromatograp...
160361,"DEUTZ, Nicolaas; ENGELEN, Marielle P; TEN HAVE...",[' Health and Kinesiology'],2012,NaN,NaN,NaN,Absence of post-prandial gut anabolism after i...,Absence of post-prandial gut anabolism after i...


In [91]:
#dept_df['Publication Date'] = dept_df['Publication Date'].str[-4:]

In [92]:
#dept_df['Title_abs'] = dept_df['Title'].fillna('') + dept_df['Abstract'].fillna('')
#dept_df = dept_df.drop(['Abstract'],axis =1)

In [20]:
dept_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160363 entries, 0 to 160362
Data columns (total 8 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Authors at my Institution              160363 non-null  object 
 1   Departments                            160363 non-null  object 
 2   Publication Date                       160363 non-null  object 
 3   Research Organizations - standardized  3046 non-null    object 
 4   Country of Research organization       3046 non-null    object 
 5   Times cited                            3046 non-null    float64
 6   Title                                  159821 non-null  object 
 7   Title_abs                              159821 non-null  object 
dtypes: float64(1), object(7)
memory usage: 9.8+ MB


In [21]:
year_wise_df = dept_df.copy()
year_wise_df.set_index('Publication Date', inplace = True)

In [22]:
year_wise_df.groupby(['Publication Date', 'Title','Title_abs','Departments','Authors at my Institution']).sum()

Times cited
Publication Date Title                                              Title_abs                                          Departments                                        Authors at my Institution              
1-01             Contemplations of the Dread and Love of God (1506) Contemplations of the Dread and Love of God (1506) [' English']                                       BOENIG, Robert E                    0.0
                 Narrative of the Life of Frederick Douglass, an... Narrative of the Life of Frederick Douglass, an... [' English']                                       DWORKIN, Ira M                      0.0
1-30             Berkeley and Spinoza                               Berkeley and Spinoza                               [' Philosophy']                                    DANIEL, Stephen H                   0.0
1926             The Moon Maid                                      The Moon Maid                                      [' International Studies']                         GOLSAN, Richard J                   0.0
1952             Historia de la revolucion mexicana.                Historia de la revolucion mexicana.                [' Philosophy']                                    BERMUDEZ OSPINA, Jose L             0.0
...                                                                                                                                                                                                           ...
2022             Working around the CLOCK: Cocaine?induced phase... Working around the CLOCK: Cocaine?induced phase... [' Neuroscience and Experimental Therapeutics']    SMITH, Laura N                      0.0
                 Zwitterionic Graphene Quantum Dots to Stabilize... Zwitterionic Graphene Quantum Dots to Stabilize... [' Chemical Engineering']                          WANG, Qingsheng                     0.0
                 scTenifoldKnk: An efficient virtual knockout to... scTenifoldKnk: An efficient virtual knockout to... ['Electrical and Computer Engineering', ' Veter... CAI, Jing                           0.0
                 “All the Other Devils this Side of Hades”: Blac... “All the Other Devils this Side of Hades”: Blac... [' History']                                       GARRETT-SCOTT, Shennette M          0.0
8-01             Overdominant epistatic loci are the primary gen... Overdominant epistatic loci are the primary gen... [' Soil and Crop Sciences']                        TABIEN, Rodante E                 220.0

[158824 rows x 1 columns]

In [66]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()


In [67]:
dept_df['Departments'] = dept_df['Departments'].str[1:-1]
for i in range(160363):
    dept_df['Departments'][i] = dept_df['Departments'][i].replace("' ","'").replace("'","")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
tmp_df = dept_df['Departments'].apply(lambda x: x.split(","))

In [69]:
tmp_df

0                                    [Atmospheric Sciences]
1                                      [Medical Physiology]
2                                    [Atmospheric Sciences]
3                          [Epidemiology and Biostatistics]
4         [Computer Science and Engineering,  Psychologi...
                                ...                        
160358                                         [Entomology]
160359                                  [Pharmacy Practice]
160360                                    [Marine Sciences]
160361                             [Health and Kinesiology]
160362                   [Psychological and Brain Sciences]
Name: Departments, Length: 160363, dtype: object

In [70]:
array_out = mlb.fit_transform(tmp_df)
df_out = pd.DataFrame(data=array_out, columns=mlb.classes_)
df_out.columns = df_out.columns.str.replace("'"," ")
dept_df = pd.concat([dept_df, df_out],axis =1).drop(['Departments'],axis =1 )
dept_df

,Authors at my Institution,Publication Date,Research Organizations - standardized,Country of Research organization,Times cited,Title,Title_abs,Accounting,Aerospace Engineering,Africana Studies,...,Teaching,Translational Medical Sciences,University Affiliated Agencies,University Libraries,Veterinary Integrative Biosciences,Veterinary Pathobiology,Veterinary Physiology and Pharmacology,Visualization,Wildlife and Fisheries Sciences,Women & Gender Studies
0,"ZHANG, Renyi",2020,Texas A&M University; California Institute of ...,United States; United States; United States; U...,652.0,Identifying airborne transmission as the domin...,Identifying airborne transmission as the domin...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"KUMAR, Rajesh",2017,NaN,NaN,NaN,Associations of fats and carbohydrate intake w...,Associations of fats and carbohydrate intake w...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"XU, Yangyang",2018,NaN,NaN,NaN,Global warming will happen faster than we think,Global warming will happen faster than we think,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"STOVER, Patrick J; JOHNSTON, Bradley C",2019,Hospital of the Holy Cross and Saint Paul; Net...,Spain; Netherlands; Canada; Canada; South Kore...,126.0,Unprocessed Red Meat and Processed Meat Consum...,Unprocessed Red Meat and Processed Meat Consum...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"DONNELLAN, Michael B; THOMAS, Shawna L",2015,NaN,NaN,NaN,Estimating the reproducibility of psychologica...,Estimating the reproducibility of psychologica...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160358,"BEHMER, Spencer T",2012,NaN,NaN,NaN,Plant sterols and host plant suitability for g...,Plant sterols and host plant suitability for g...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160359,"KHASAWNEH, Fadi",2011,NaN,NaN,NaN,Glutathione and Adaptive Immune Responses agai...,Glutathione and Adaptive Immune Responses agai...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160360,"KAISER, Karl",2011,NaN,NaN,NaN,Characterization of Lignin by Gas Chromatograp...,Characterization of Lignin by Gas Chromatograp...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160361,"DEUTZ, Nicolaas; ENGELEN, Marielle P; TEN HAVE...",2012,NaN,NaN,NaN,Absence of post-prandial gut anabolism after i...,Absence of post-prandial gut anabolism after i...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
tmp_df = dept_df['Country of Research organization'].astype(str).apply(lambda x: x.split(";"))
array_out = mlb.fit_transform(tmp_df)
df_out = pd.DataFrame(data=array_out, columns=mlb.classes_)
df_out.columns = df_out.columns.str.replace("'"," ")
dept_df = pd.concat([dept_df, df_out],axis =1).drop(['Country of Research organization'],axis =1 )
dept_df

,Authors at my Institution,Publication Date,Research Organizations - standardized,Times cited,Title,Title_abs,Accounting,Aerospace Engineering,Africana Studies,Aggie STEM,...,Switzerland,Taiwan,Thailand,Turkey,United Kingdom,United States,Uruguay,Uzbekistan,Yemen,nan
0,"ZHANG, Renyi",2020,Texas A&M University; California Institute of ...,652.0,Identifying airborne transmission as the domin...,Identifying airborne transmission as the domin...,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,"KUMAR, Rajesh",2017,NaN,NaN,Associations of fats and carbohydrate intake w...,Associations of fats and carbohydrate intake w...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"XU, Yangyang",2018,NaN,NaN,Global warming will happen faster than we think,Global warming will happen faster than we think,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,"STOVER, Patrick J; JOHNSTON, Bradley C",2019,Hospital of the Holy Cross and Saint Paul; Net...,126.0,Unprocessed Red Meat and Processed Meat Consum...,Unprocessed Red Meat and Processed Meat Consum...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"DONNELLAN, Michael B; THOMAS, Shawna L",2015,NaN,NaN,Estimating the reproducibility of psychologica...,Estimating the reproducibility of psychologica...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160358,"BEHMER, Spencer T",2012,NaN,NaN,Plant sterols and host plant suitability for g...,Plant sterols and host plant suitability for g...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
160359,"KHASAWNEH, Fadi",2011,NaN,NaN,Glutathione and Adaptive Immune Responses agai...,Glutathione and Adaptive Immune Responses agai...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
160360,"KAISER, Karl",2011,NaN,NaN,Characterization of Lignin by Gas Chromatograp...,Characterization of Lignin by Gas Chromatograp...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
160361,"DEUTZ, Nicolaas; ENGELEN, Marielle P; TEN HAVE...",2012,NaN,NaN,Absence of post-prandial gut anabolism after i...,Absence of post-prandial gut anabolism after i...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [74]:
tmp_df = dept_df['Research Organizations - standardized'].astype(str).apply(lambda x: x.split(";"))
array_out = mlb.fit_transform(tmp_df)
df_out = pd.DataFrame(data=array_out, columns=mlb.classes_)
df_out.columns = df_out.columns.str.replace("'"," ")
dept_df = pd.concat([dept_df, df_out],axis =1).drop(['Research Organizations - standardized'],axis =1 )
dept_df

,Authors at my Institution,Publication Date,Times cited,Title,Title_abs,Accounting,Aerospace Engineering,Africana Studies,Aggie STEM,Agricultural Economics,...,Xi an Jiaotong University,Yale University,Yanbian University,Yerevan State University,Yeungnam University,Zagaya,Zhejiang University,Zoological Research Museum Alexander Koenig,nan,École Normale Supérieure de Lyon
0,"ZHANG, Renyi",2020,652.0,Identifying airborne transmission as the domin...,Identifying airborne transmission as the domin...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"KUMAR, Rajesh",2017,NaN,Associations of fats and carbohydrate intake w...,Associations of fats and carbohydrate intake w...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,"XU, Yangyang",2018,NaN,Global warming will happen faster than we think,Global warming will happen faster than we think,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,"STOVER, Patrick J; JOHNSTON, Bradley C",2019,126.0,Unprocessed Red Meat and Processed Meat Consum...,Unprocessed Red Meat and Processed Meat Consum...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"DONNELLAN, Michael B; THOMAS, Shawna L",2015,NaN,Estimating the reproducibility of psychologica...,Estimating the reproducibility of psychologica...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160358,"BEHMER, Spencer T",2012,NaN,Plant sterols and host plant suitability for g...,Plant sterols and host plant suitability for g...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
160359,"KHASAWNEH, Fadi",2011,NaN,Glutathione and Adaptive Immune Responses agai...,Glutathione and Adaptive Immune Responses agai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
160360,"KAISER, Karl",2011,NaN,Characterization of Lignin by Gas Chromatograp...,Characterization of Lignin by Gas Chromatograp...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
160361,"DEUTZ, Nicolaas; ENGELEN, Marielle P; TEN HAVE...",2012,NaN,Absence of post-prandial gut anabolism after i...,Absence of post-prandial gut anabolism after i...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
(dept_df.iloc[:,5:].sum(axis=0)>1).values.sum()